In [17]:
import pandas as pd

# Read 200kb bins file
binbed = pd.read_csv(
    # "../bin_200kb_all.bed",
    "../../../../mosaicatcher-update/workflow/data/bin_200kb_all.bed",
    sep="\t",
    names=["chrom", "start", "end", "bin_id"],
)
binbed["ID"] = binbed["chrom"] + "_" + binbed["start"].astype(str) + "_" + binbed["end"].astype(str)

# Turn chrom into categorical
binbed["chrom"] = pd.Categorical(
    binbed["chrom"],
    categories=["chr{}".format(e) for e in range(1, 23)] + ["chrX", "chrY"],
    ordered=True,
)

# Sort & filter out chrY #TMP / can be changed
binbed = binbed.sort_values(by=["chrom", "start", "end"]).reset_index(drop=True)
binbed["w"], binbed["c"], binbed["class"]= 0, 0, None
# binbed = binbed.loc[~binbed["chrom"].isin(["chrY"])]
binbed



,chrom,start,end,bin_id,ID,w,c,class
0,chr1,0,200000,bin_200kb_1,chr1_0_200000,0,0,None
1,chr1,200000,400000,bin_200kb_2,chr1_200000_400000,0,0,None
2,chr1,400000,600000,bin_200kb_3,chr1_400000_600000,0,0,None
3,chr1,600000,800000,bin_200kb_4,chr1_600000_800000,0,0,None
4,chr1,800000,1000000,bin_200kb_5,chr1_800000_1000000,0,0,None
...,...,...,...,...,...,...,...,...
15448,chrY,56400000,56600000,bin_200kb_15426,chrY_56400000_56600000,0,0,None
15449,chrY,56600000,56800000,bin_200kb_15427,chrY_56600000_56800000,0,0,None
15450,chrY,56800000,57000000,bin_200kb_15428,chrY_56800000_57000000,0,0,None
15451,chrY,57000000,57200000,bin_200kb_15429,chrY_57000000_57200000,0,0,None


In [5]:

# Read SV file
df = pd.read_csv("../../../../mosaicatcher-update/.tests/data_CHR17/RPE-BM510/counts/RPE-BM510.txt.raw.gz", sep="\t")
# df = pd.read_csv("../stringent_filterTRUE.tsv", sep="\t")
df["ID"] = df["chrom"] + "_" + df["start"].astype(str) + "_" + df["end"].astype(str)
df
# Read 2

,chrom,start,end,sample,cell,c,w,class,ID
0,chr17,0,200000,RPE-BM510,BM510x04_PE20301,0,11,WW,chr17_0_200000
1,chr17,200000,400000,RPE-BM510,BM510x04_PE20301,0,21,WW,chr17_200000_400000
2,chr17,400000,600000,RPE-BM510,BM510x04_PE20301,0,19,WW,chr17_400000_600000
3,chr17,600000,800000,RPE-BM510,BM510x04_PE20301,0,19,WW,chr17_600000_800000
4,chr17,800000,1000000,RPE-BM510,BM510x04_PE20301,0,35,WW,chr17_800000_1000000
...,...,...,...,...,...,...,...,...,...
7501,chr17,82400000,82600000,RPE-BM510,BM510x04_PE20319,16,18,WC,chr17_82400000_82600000
7502,chr17,82600000,82800000,RPE-BM510,BM510x04_PE20319,25,30,WC,chr17_82600000_82800000
7503,chr17,82800000,83000000,RPE-BM510,BM510x04_PE20319,17,29,WC,chr17_82800000_83000000
7504,chr17,83000000,83200000,RPE-BM510,BM510x04_PE20319,18,16,WC,chr17_83000000_83200000


In [18]:
l = list()
for cell in df.cell.unique().tolist():
    tmp_df = pd.concat(
            [
                binbed.loc[~binbed["ID"].isin(df.loc[df["cell"] == cell].ID.values.tolist())],
                df.loc[df["cell"] == cell]
            ]
        )
    tmp_df["cell"] = cell
    tmp_df["sample"] =  df.loc[df["cell"] == cell, "sample"].values.tolist()[0]
    l.append(
        tmp_df
    )
populated_df = pd.concat(l).sort_values(by=["cell", "chrom", "start"])
populated_df.to_csv("test.txt.gz", compression="gzip", sep="\t", index=False)
